In [1]:
import numpy as np
import random
from collections import deque, defaultdict
import queue

In [2]:
#ANSI color codes for red and black text, and RESET will reset the color back to normal after displaying the card
RED = "\033[91m"
BLACK = "\033[90m"
RESET = "\033[0m"

#for each card type, we color code it and give it the appropriate symbol: RH=red heart, RD=red diamond, BS=black spade, BC=black club, blank=blank space
suits_symbols = {
    "RH": (RED, "❤️"),
    "RD": (RED, "♦️"),
    "BS": (BLACK, "♠️"),
    "BC": (BLACK, "♣️"),
    "blank": (RESET, "⬜")
}

#Initializes the game board (which is the final goal state) and blank board(which will be what the computer 'flips' to try get to the game board state)
#we define the suits and the faces to make a combination of them, a total of 28 combinations. We then multiply that by two to get our full 
#deck and therefore be able to make a pair match between identical cards
#the deck is then randomly shuffled and put into a 7x8 2d array --> this is the final goal state and what we want to use our algorithms for to 'flip' the
#cards from an initial blank 'unflipped' state
def initial_boards():
    suits = ["RH", "RD", "BS", "BC"]
    faces = ["A", "2", "3", "4", "5", "6", "7"]
    combinations = [suit + face for suit in suits for face in faces]
    deck = combinations * 2  # Duplicate to get 56 cards, to get pair matches
    random.shuffle(deck)
    game_board = np.array(deck).reshape(7, 8)
    blank_board = np.full((7, 8), "blank")
    return game_board, blank_board

#this is the function for visualizing the cards, where the first string is made up of the suit (RH,RD,BS,BC) and the second string is
#the remaining face value (A,2,3,4,5,6,7) that displays the appropriate color and symbol
def visualize_card(card):
    suit = card[:2] if card != "blank" else "blank"
    face = card[2:] if card != "blank" else ""
    color, symbol = suits_symbols[suit]
    return f"{color}{card if card != 'blank' else '  '} {symbol}{RESET}"

# Function to print boards
def print_boards(game_board, blank_board):
    print("Game Board (revealed cards, goal state):\n", game_board)
    print("\nBlank Board (flipped board):\n" )
    for row in blank_board:
        print(" ".join(visualize_card(card) for card in row))
    print()


In [3]:
#BFS function
def bfs_match_all_pairs(game_board, blank_board):
    
    #takes the values for the rows and columns of the game board, creates a set for matched positions
    rows, cols = game_board.shape
    matched_positions = set() 

    #stores the locations of the cards on game_board where card values are keys and grid positions are values 
    card_positions = defaultdict(list)
    for r in range(rows):
        for c in range(cols):
            card_positions[game_board[r, c]].append((r, c))

    #for each card, for two card positions, we check if they are a match. the target position is the second, matching pair to pos1
    #we queue the cards and start doing the bfs search (where we search each cell's neighbors) to find the match, and if a match is found we stop the search for that pair
    #if the match is found, we print that it was founded, and we add the cell to be visited and break from the queue. we also add the 
    #two positions to be a matched position in the blank board.
    for card, positions in card_positions.items():
        if len(positions) == 2:
            pos1, pos2 = positions

            if pos1 not in matched_positions and pos2 not in matched_positions:
                queue = [pos1]
                visited = set(queue)
                target_pos = pos2
                found = False

                while queue:
                    current = queue.pop(0)
                    row, col = current
                    print(f"BFS Searching for {visualize_card(card)} at position ({row}, {col})")

                    revealed_card = game_board[row, col]
                    print(f"Revealed card at ({row}, {col}): {visualize_card(revealed_card)}")
                    
                    if current == target_pos:
                        found = True
                        print(f"Match found for {visualize_card(card)} at positions {pos1} and {pos2}!\n")
                        break

                    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        r, c = row + dr, col + dc
                        if 0 <= r < rows and 0 <= c < cols and (r, c) not in visited:
                            visited.add((r, c))
                            queue.append((r, c))

                if found:
                    r1, c1 = pos1
                    r2, c2 = pos2
                    blank_board[r1, c1] = game_board[r1, c1]
                    blank_board[r2, c2] = game_board[r2, c2]
                    matched_positions.add(pos1)
                    matched_positions.add(pos2)
                else:
                    print(f"No match found for {visualize_card(card)} at positions {pos1} and {pos2}.\n")

    return blank_board


In [4]:
def dfs_match_all_pairs(game_board, blank_board):
    rows, cols = game_board.shape
    matched_positions = set()

    #records the positions of the cards
    card_positions = defaultdict(list)
    for r in range(rows):
        for c in range(cols):
            card_positions[game_board[r, c]].append((r, c))

    #searches the cards using DFS, keeps track if the cards were visited, goal is to find the second match, the goal/target position
    for card, positions in card_positions.items():
        if len(positions) == 2:
            pos1, pos2 = positions
            
            
            if pos1 not in matched_positions and pos2 not in matched_positions:
                stack = [pos1] 
                visited = set(stack)   
                target_pos = pos2 
                found = False

                #takes the top of the stack, and searches for it's match along a path from neighboring cells
                while stack:
                    current = stack.pop()
                    row, col = current
                    print(f"DFS Searching for {visualize_card(card)} at position ({row}, {col})")

                    revealed_card = game_board[row, col]
                    print(f"Revealed card at ({row}, {col}): {visualize_card(revealed_card)}")

                    if current == target_pos:
                        found = True
                        print(f"Match found for {visualize_card(card)} at positions {pos1} and {pos2}!\n")
                        break

                    for dr, dc in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
                        r, c = row + dr, col + dc
                        if 0 <= r < rows and 0 <= c < cols and (r, c) not in visited:
                            visited.add((r, c))
                            stack.append((r, c))
                if found:
                    r1, c1 = pos1
                    r2, c2 = pos2
                    blank_board[r1, c1] = game_board[r1, c1]
                    blank_board[r2, c2] = game_board[r2, c2]
                    matched_positions.add(pos1)
                    matched_positions.add(pos2)

                else:
                    print(f"No match found for {visualize_card(card)} at positions {pos1} and {pos2}.\n")

    return blank_board


In [5]:
#prints the boards and utilizes the algorithms, as well as printing the final states of the boards
#bfs checks for the match from adjacent cells, layer by layer
#dfs checks for the match from adjacent cells, one path at a time
#the patterns/paths in which each search takes can be shown from the code above

def auto_solve_game_BFS():
    game_board, blank_board = initial_boards()
    print("Initial Boards :")
    print_boards(game_board, blank_board)
    
    final_blank_board = bfs_match_all_pairs(game_board, blank_board)

    print("\nFinal Board State :")
    print_boards(game_board, final_blank_board)

def auto_solve_game_DFS():
    game_board, blank_board = initial_boards()
    print("Initial Boards:")
    print_boards(game_board, blank_board)
    
    final_blank_board = dfs_match_all_pairs(game_board, blank_board)
    
    print("\nFinal Board State :")
    print_boards(game_board, final_blank_board)


In [6]:
auto_solve_game_BFS()

Initial Boards :
Game Board (revealed cards, goal state):
 [['BC3' 'RD3' 'RD6' 'BC5' 'RH3' 'RH6' 'BS6' 'BC7']
 ['RD4' 'BS7' 'RH2' 'BS5' 'BS4' 'RD6' 'BS6' 'RH3']
 ['BC2' 'RD5' 'BC6' 'BC2' 'BC3' 'RH5' 'BS7' 'RD2']
 ['BSA' 'RH6' 'RD7' 'RH4' 'BS4' 'BC6' 'BS2' 'BS3']
 ['BCA' 'RDA' 'BSA' 'BC7' 'BC4' 'RH4' 'BCA' 'BS5']
 ['RD3' 'BC5' 'BS3' 'RH7' 'RH2' 'RD5' 'RD2' 'RHA']
 ['RD7' 'RH5' 'BS2' 'RH7' 'BC4' 'RDA' 'RD4' 'RHA']]

Blank Board (flipped board):

   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜

BFS Searching for BC3 ♣️ at position (0, 0)
Revealed card at (0, 0): BC3 ♣️
BFS Searching for BC3 ♣️ at position (1, 0)
Revealed card at (1, 0): RD4 ♦️
BFS Searching for BC3 ♣️ at position (0, 1)
Revealed card at (0, 1): RD3 ♦️
BFS Searching for BC3 ♣️ at position (2, 0)

In [7]:
auto_solve_game_DFS()

Initial Boards:
Game Board (revealed cards, goal state):
 [['BS3' 'RD6' 'RD5' 'BS7' 'RH2' 'RH6' 'BC4' 'BC7']
 ['RH4' 'BC6' 'BC6' 'RD2' 'BC2' 'BC4' 'RD3' 'RDA']
 ['RDA' 'RD4' 'RD7' 'BSA' 'BS6' 'BC3' 'RD4' 'BCA']
 ['RHA' 'BS6' 'RH5' 'BS5' 'RD2' 'BS5' 'BS7' 'RH5']
 ['RD5' 'BC3' 'RH6' 'BS2' 'BC5' 'BS2' 'RD7' 'BSA']
 ['BS3' 'RD6' 'RH3' 'RH4' 'RH7' 'BS4' 'RH3' 'RH2']
 ['BS4' 'RH7' 'BC7' 'BC5' 'BC2' 'BCA' 'RD3' 'RHA']]

Blank Board (flipped board):

   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜

DFS Searching for BS3 ♠️ at position (0, 0)
Revealed card at (0, 0): BS3 ♠️
DFS Searching for BS3 ♠️ at position (0, 1)
Revealed card at (0, 1): RD6 ♦️
DFS Searching for BS3 ♠️ at position (0, 2)
Revealed card at (0, 2): RD5 ♦️
DFS Searching for BS3 ♠️ at position (0, 3)


Iterative deepening search

In [8]:
# Iterative deepening search (IDS)
def depth_limited_search(game_data, blank_array, start, depth_limit):
    target_card = game_data[start]  # Card we are looking to match 
    rows, cols = game_data.shape 
    flips = []

    # Goes through the card positions
    for row in range(rows):
        for col in range(cols):
            if (row, col) != start and blank_array[row, col] == "blank":
                flips.append((row, col))
                # Checking if we've reached the depth limit
                if len(flips) > depth_limit:
                    return None  

                # Checking if this card matches the starting card
                if game_data[row, col] == target_card:
                    return flips  # flips the card if we've found a match

    return None  # if no match is found within the depth limit

# IDS to find all matches
def iterative_deepening_search(game_data):
    blank_array = np.full((7, 8), "blank")
    attempts = 0
    rows, cols = game_data.shape
    matched_positions = set()

    # Loops over unmatched cards
    for row in range(rows):
        for col in range(cols):
            if (row, col) not in matched_positions:
                depth = 1
                found_match = False

                # Increases depth until a match is found
                while not found_match:
                    flips = depth_limited_search(game_data, blank_array, (row, col), depth)

                    if flips is not None:
                        # Checking each flipped position to verify the match
                        for flip in flips:
                            if game_data[flip] == game_data[row, col]:
                                blank_array[row, col] = game_data[row, col]
                                blank_array[flip] = game_data[flip]
                                matched_positions.add((row, col))
                                matched_positions.add(flip)
                                found_match = True
                                print(f"Found match for {visualize_card(game_data[row, col])} at {(row, col)} and {flip} with depth {depth}")
                                print_boards(game_data, blank_array)
                                print("")
                                break
                    else:
                        depth += 1  # if no match is found it increases the depth with one

                    attempts += 1

# Generates a game board for testing
def simulate_game_data():
    game_board, _ = initial_boards()
    return game_board

# Run the game with IDS
game_data = simulate_game_data()
print("Running Iterative Deepening Search for Matches:\n")
iterative_deepening_search(game_data)

Running Iterative Deepening Search for Matches:

Found match for BC2 ♣️ at (0, 0) and (3, 1) with depth 25
Game Board (revealed cards, goal state):
 [['BC2' 'BS7' 'RD3' 'RH6' 'RH3' 'RD6' 'BCA' 'BS4']
 ['BC7' 'RH6' 'RD4' 'RD5' 'BS5' 'BC5' 'RD4' 'BC3']
 ['BS3' 'RH5' 'RDA' 'RD5' 'RDA' 'RH3' 'RH4' 'BC3']
 ['RH2' 'BC2' 'RH7' 'RD3' 'BC4' 'BSA' 'BS2' 'RH7']
 ['RH5' 'BS5' 'RH4' 'BC7' 'RD7' 'BS3' 'RHA' 'RD2']
 ['RD6' 'BS6' 'BCA' 'BC4' 'BS2' 'RD7' 'BS7' 'BC6']
 ['RHA' 'BC5' 'RH2' 'BS6' 'BS4' 'RD2' 'BSA' 'BC6']]

Blank Board (flipped board):

BC2 ♣️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜ BC2 ♣️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜


Found match for BS7 ♠️ at (0, 1) and (5, 6) with depth 44
Game Board (revealed cards, goal state):
 [['BC2' 'BS7' 'RD3' 'RH6' 'RH3' 'RD6' 'BCA' 'BS4']
 ['BC7' 'RH6' 'RD4' 'RD5'

Greedy Search

In [9]:
def heuristic(nodeA, nodeB):
    (xA, yA) = nodeA
    (xB, yB) = nodeB
    return abs(xA - xB) + abs(yA - yB)

# Greedy Search to find and match pairs of cards
def greedy_search_match(game_data):
    blank_array = np.full((7, 8), "blank")  # Initialize blank board with "blank" placeholders
    explored = set()  # Keeps track of card matches

    # Loops through the grid
    #If explored, skips
    # Otherwise, sets card as target card and adds position to explore
    for row in range(game_data.shape[0]):
        for col in range(game_data.shape[1]):
            if (row, col) not in explored:
                target_card = game_data[row, col]
                explored.add((row, col))

                # Finding the closest match to the target
                frontier = queue.PriorityQueue()
                for r in range(game_data.shape[0]):
                    for c in range(game_data.shape[1]):
                        if (r, c) != (row, col) and (r, c) not in explored:
                            if game_data[r, c] == target_card:
                                priority = heuristic((row, col), (r, c))
                                frontier.put((priority, (r, c)))

                # Revealing the closest match
                if not frontier.empty():
                    closest_match = frontier.get()[1]
                    r, c = closest_match
                    blank_array[row, col] = game_data[row, col]
                    blank_array[r, c] = game_data[r, c]
                    explored.add((r, c))
                    print(f"Greedy match: Found pair {visualize_card(target_card)} at ({row},{col}) and ({r},{c})")
                    print_boards(game_data, blank_array)
                    print("")

                frontier.queue.clear()  # Clear queue for next iteration

# Generates a game board for testing
def simulate_game_data():
    game_board, _ = initial_boards()
    return game_board

# Run the game with Greedy Search for Matches
game_data = simulate_game_data()
print("Running Greedy Search for Matches:\n")
greedy_search_match(game_data)

Running Greedy Search for Matches:

Greedy match: Found pair RD6 ♦️ at (0,0) and (3,1)
Game Board (revealed cards, goal state):
 [['RD6' 'BS3' 'RD4' 'BS7' 'RD7' 'BS6' 'RH7' 'RD5']
 ['BC7' 'RHA' 'BCA' 'BC7' 'RH6' 'BSA' 'RH4' 'RH4']
 ['RH5' 'RD5' 'BSA' 'BC2' 'BC6' 'RHA' 'BC2' 'RH2']
 ['BS2' 'RD6' 'RD3' 'RD2' 'BC5' 'BC3' 'RH5' 'RH3']
 ['RH6' 'RD2' 'BS6' 'RDA' 'BS5' 'BS3' 'RH3' 'BS5']
 ['RH2' 'BCA' 'BS4' 'RD3' 'BC6' 'BC4' 'BS2' 'RD4']
 ['RH7' 'BC3' 'BC5' 'BC4' 'RDA' 'BS4' 'BS7' 'RD7']]

Blank Board (flipped board):

RD6 ♦️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜ RD6 ♦️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜


Greedy match: Found pair BS3 ♠️ at (0,1) and (4,5)
Game Board (revealed cards, goal state):
 [['RD6' 'BS3' 'RD4' 'BS7' 'RD7' 'BS6' 'RH7' 'RD5']
 ['BC7' 'RHA' 'BCA' 'BC7' 'RH6' 'BSA' 'RH4' 'RH4']
 

A* Search 

In [10]:
def heuristic(nodeA, nodeB):
    (xA, yA) = nodeA
    (xB, yB) = nodeB
    return abs(xA - xB) + abs(yA - yB)

# A* Search to find and match pairs of cards
def astar_search_match(game_data):
    blank_array = np.full((7, 8), "blank")  # Start with a blank array of cards
    explored = set()  # Track matched cards
    attempts = 0  # Count the number of attempts

    # Loop through each position on the game board
    for row in range(game_data.shape[0]):
        for col in range(game_data.shape[1]):
            if (row, col) not in explored:  # If position hasn't been matched yet
                target_card = game_data[row, col]
                explored.add((row, col))

                # Priority queue to find the closest match to the target card
                frontier = queue.PriorityQueue()
                for r in range(game_data.shape[0]):
                    for c in range(game_data.shape[1]):
                        if (r, c) != (row, col) and (r, c) not in explored:
                            if game_data[r, c] == target_card:  # If cards match
                                priority = heuristic((row, col), (r, c))  # Calculate priority based on distance
                                frontier.put((priority, (r, c)))  # Add to queue with priority

                # Reveal the closest match if available
                if not frontier.empty():
                    closest_match = frontier.get()[1]
                    r, c = closest_match
                    blank_array[row, col] = game_data[row, col]  # Reveal target card on blank board
                    blank_array[r, c] = game_data[r, c]  # Reveal matched card on blank board
                    explored.add((r, c))  # Add matched positions to explored
                    print(f"A* match: Found pair {visualize_card(target_card)} at ({row},{col}) and ({r},{c})")
                    print_boards(game_data, blank_array)
                    print("")

                attempts += 1  # Increase attempts count with one
                frontier.queue.clear()  # Clear queue for the next target

# Run the game with A* Search for Matches
game_data = simulate_game_data()
print("Running A* Search for Matches:\n")
astar_search_match(game_data)

Running A* Search for Matches:

A* match: Found pair BC6 ♣️ at (0,0) and (5,3)
Game Board (revealed cards, goal state):
 [['BC6' 'RH3' 'BS2' 'RD4' 'BC3' 'RD7' 'BS5' 'BS3']
 ['BS7' 'BSA' 'RD6' 'BC4' 'RD7' 'BCA' 'RH4' 'BC3']
 ['RH6' 'RD2' 'RD3' 'BS5' 'BC7' 'BS4' 'RHA' 'RHA']
 ['BS2' 'BS7' 'BS6' 'RD3' 'RH2' 'BS6' 'BSA' 'BC4']
 ['RH5' 'RD5' 'BC7' 'RH7' 'RD4' 'BS4' 'BC5' 'RH4']
 ['BC2' 'RD5' 'RH5' 'BC6' 'RH7' 'BS3' 'BC5' 'RD2']
 ['RD6' 'BC2' 'BCA' 'RDA' 'RDA' 'RH6' 'RH3' 'RH2']]

Blank Board (flipped board):

BC6 ♣️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜ BC6 ♣️    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜


A* match: Found pair RH3 ❤️ at (0,1) and (6,6)
Game Board (revealed cards, goal state):
 [['BC6' 'RH3' 'BS2' 'RD4' 'BC3' 'RD7' 'BS5' 'BS3']
 ['BS7' 'BSA' 'RD6' 'BC4' 'RD7' 'BCA' 'RH4' 'BC3']
 ['RH6' 'RD2'

Deep-limited search (DLS)

In [11]:
# Deep-limited search (DLS)
def depth_limited_search(game_board, blank_board, max_depth=100):
    # Stack for DLS
    stack = [(blank_board, [], 0)]  # (current_board, flipped_cards, depth)
    explored = set()  # To store explored states
    steps = 0

    while stack:
        current_board, flipped_cards, depth = stack.pop()

        # If we have reached the maximum depth, stop searching
        if depth >= max_depth:
            continue

        # Convert current_board to a tuple of tuples for hashing
        current_board_tuple = tuple(map(tuple, current_board))

        # Check if current board is already explored
        if current_board_tuple in explored:
            continue

        explored.add(current_board_tuple)

        # Check if the game is won
        if np.all(current_board != "blank"):
            print("Game won!")
            return current_board, steps

        # Try to flip a new card
        for i in range(current_board.shape[0]):
            for j in range(current_board.shape[1]):
                if current_board[i, j] == "blank":
                    # Try flipping this card and pair it with another one
                    for k in range(current_board.shape[0]):
                        for l in range(current_board.shape[1]):
                            if current_board[k, l] == "blank" and (i, j) != (k, l):
                                new_board = current_board.copy()
                                new_flipped_cards = flipped_cards + [((i, j), (k, l))]
                                new_board[i, j], new_board[k, l] = game_board[i, j], game_board[k, l]

                                # If we found a match, stop searching this pair
                                if game_board[i, j] == game_board[k, l]:
                                    stack.append((new_board, new_flipped_cards, depth + 1))
                                    steps += 1
                                    print_boards(game_board, new_board)

    print("No solution found.")
    return None, steps




# Main test run
game_data, blank_board = initial_boards()
print("Starting the Memory Game with Depth-Limited Search (DLS)...")
result_dls, steps_dls = depth_limited_search(game_data, blank_board)
print("\nDLS Result:", "Success" if np.all(result_dls != "blank") else "Failure")
print("Steps:", steps_dls)




Starting the Memory Game with Depth-Limited Search (DLS)...
Game Board (revealed cards, goal state):
 [['BS7' 'BC4' 'RD6' 'RHA' 'RD4' 'BC2' 'BC2' 'RH5']
 ['RD2' 'BCA' 'RD5' 'RD4' 'BS2' 'BC6' 'RD3' 'BC3']
 ['BS4' 'RH7' 'BCA' 'RH3' 'BS3' 'BC5' 'BC6' 'BSA']
 ['RD7' 'BS2' 'BS3' 'BC7' 'BS4' 'BS5' 'RH7' 'RDA']
 ['RH2' 'RH4' 'RH4' 'RH5' 'RDA' 'RHA' 'BS5' 'RH2']
 ['BC3' 'RD2' 'BSA' 'RD6' 'RD5' 'BC4' 'BS6' 'BS7']
 ['BS6' 'RH6' 'RD3' 'RH3' 'RH6' 'RD7' 'BC5' 'BC7']]

Blank Board (flipped board):

BS7 ♠️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜ BS7 ♠️
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜

Game Board (revealed cards, goal state):
 [['BS7' 'BC4' 'RD6' 'RHA' 'RD4' 'BC2' 'BC2' 'RH5']
 ['RD2' 'BCA' 'RD5' 'RD4' 'BS2' 'BC6' 'RD3' 'BC3']
 ['BS4' 'RH7' 'BCA' 'RH3' 'BS3' 'BC5' 'BC6' 'BSA']
 ['RD7' 'BS2' 'BS3' 'BC7' 'BS

Uniform cost search (UCS)

In [12]:
# Uniform Cost Search (UCS)
def uniform_cost_search(game_board, blank_board):
    pq = queue.PriorityQueue()
    pq.put((0, blank_board, []))  # (cost, current_board, flipped_cards)
    explored = set()
    steps = 0

    while not pq.empty():
        cost, current_board, flipped_cards = pq.get()

        # Convert current_board to a tuple of tuples for hashing
        current_board_tuple = tuple(map(tuple, current_board))

        # If the board has already been explored, skip it
        if current_board_tuple in explored:
            continue

        explored.add(current_board_tuple)

        # If the game is won, return the solution
        if np.all(current_board != "blank"):
            print("Game won!")
            return current_board, steps

        # Try flipping a new card
        for i in range(current_board.shape[0]):
            for j in range(current_board.shape[1]):
                if current_board[i, j] == "blank":
                    for k in range(current_board.shape[0]):
                        for l in range(current_board.shape[1]):
                            if current_board[k, l] == "blank" and (i, j) != (k, l):
                                new_board = current_board.copy()
                                new_flipped_cards = flipped_cards + [((i, j), (k, l))]
                                new_board[i, j], new_board[k, l] = game_board[i, j], game_board[k, l]

                                # If we found a match, move to the next state
                                if game_board[i, j] == game_board[k, l]:
                                    # new_board 
                                    pq.put((cost + 1, tuple(map(tuple, new_board)), new_flipped_cards))

                                    steps += 1
                                    print_boards(game_board, new_board)

    print("No solution found.")
    return None, steps
game_data, blank_board = initial_boards()
print("\nStarting the Memory Game with Uniform Cost Search (UCS)...")
result_ucs, steps_ucs = uniform_cost_search(game_data, blank_board)
print("\nUCS Result:", "Success" if result_ucs else "Failure")
print("Steps:", steps_ucs)


Starting the Memory Game with Uniform Cost Search (UCS)...
Game Board (revealed cards, goal state):
 [['BS2' 'BC4' 'BS4' 'RD6' 'BS6' 'BS3' 'RD5' 'RHA']
 ['BSA' 'RH2' 'BS7' 'BC6' 'BS4' 'BS2' 'RD7' 'RH4']
 ['RDA' 'RDA' 'BS3' 'BCA' 'RH7' 'RD4' 'BC5' 'BSA']
 ['BC2' 'BS7' 'RH5' 'BS6' 'BC3' 'BS5' 'RH3' 'RD3']
 ['RD5' 'BCA' 'BC2' 'RD7' 'BC5' 'RD6' 'RH6' 'RH2']
 ['BC7' 'BS5' 'BC6' 'RH4' 'RHA' 'BC4' 'BC7' 'RH7']
 ['RD3' 'RH5' 'RH3' 'BC3' 'RD4' 'RH6' 'RD2' 'RD2']]

Blank Board (flipped board):

BS2 ♠️    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜ BS2 ♠️    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜
   ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜    ⬜

Game Board (revealed cards, goal state):
 [['BS2' 'BC4' 'BS4' 'RD6' 'BS6' 'BS3' 'RD5' 'RHA']
 ['BSA' 'RH2' 'BS7' 'BC6' 'BS4' 'BS2' 'RD7' 'RH4']
 ['RDA' 'RDA' 'BS3' 'BCA' 'RH7' 'RD4' 'BC5' 'BSA']
 ['BC2' 'BS7' 'RH5' 'BS6' 'BC